In [5]:
!pip freeze | grep scikit-learn

scikit-learn==1.4.2


In [6]:
!python -V

Python 3.12.3


In [7]:
import pickle
import pandas as pd
import numpy as np
import datetime

In [8]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/Users/isabellevea/anaconda3/envs/mlops-zoomcamp/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/isabellevea/anaconda3/envs/mlops-zoomcamp/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [20]:
year = 2023
month = 3
taxi_type = 'yellow'

input_file=f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'

output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'

categorical = ['PULocationID', 'DOLocationID']


In [14]:
def apply_model(input_file, categorical, output_file):
    df = read_data(input_file)
    dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(dicts)
    y_pred = model.predict(X_val)

    print(f'the standard deviation of the our predictions is {np.std(y_pred)}')

    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    df['predictions'] = y_pred
    df_result = df[['ride_id','predictions']] 

    df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
    )


def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [21]:
apply_model(input_file=input_file, categorical=categorical, output_file=output_file)

the standard deviation of the our predictions is 6.247488852238703


In [16]:
!mkdir output

In [19]:
!mkdir output/yellow